In [1]:
!pip install billboard.py

In [1]:
import billboard
import requests

I'll try going back and finding historical charts/seeing if there's another type of chart that's more helpful. But first let's look at the current hot 100 and try to match the songs up with genres.

In [3]:
chart = billboard.ChartData('hot-100')

What's the number one song?

In [4]:
chart[0]

'Cheap Thrills' by Sia Featuring Sean Paul

Okay, let's look for the Spotify ID.

In [8]:
chart[0].spotifyID

'7BPw9h8vCIX94N0rjFU12S'

In [11]:
songid = chart[0].spotifyID

Now, I'm going to try to use that to find the genre.

endpoints: https://api.spotify.com /v1/tracks/{id} or /v1/tracks?ids={ids} 

In [12]:
response = requests.get('https://api.spotify.com/v1/tracks/' + songid)

data = response.json()

In [15]:
data.keys()

dict_keys(['type', 'duration_ms', 'href', 'name', 'popularity', 'preview_url', 'explicit', 'id', 'external_urls', 'disc_number', 'track_number', 'external_ids', 'artists', 'available_markets', 'uri', 'album'])

Oh no! Track objects don't have genres, only albums do...

In [17]:
data['album']['id']

'3Xz1Bcn4YP9bdqlsstMfuJ'

In [19]:
albumid = data['album']['id']

In [20]:
response = requests.get('https://api.spotify.com/v1/albums/' + albumid)

data = response.json()

In [24]:
data['genres']

[]

Uh oh... let's see how many of the top 100 don't have genres...

In [25]:
top100 = []
for item in chart:
    songid = chart[0].spotifyID
    response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
    data = response.json()
    albumid = data['album']['id']
    response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
    data = response.json()
    top100.append(data['genres'])

KeyError: 'genres'

In [26]:
top100

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

Damn. Okay, it seems like it's too many requests from the Spotify API *and* Spotify doesn't have much genre info, at least not for the latest and greatest music. 
<p>What I'm going to do:
* try it with a list 1 year old
* if that doesn't work, look for one of the liner notes datasets or maybe that Google one

In [27]:
chart = billboard.ChartData('hot-100', date = '2015-07-01')

In [49]:
top100 = []
for item in chart[10:20]:
    songid = item.spotifyID
    response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
    data = response.json()
    albumid = data['album']['id']
    response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
    data = response.json()
    top100.append(data['genres'])

In [50]:
top100

[[], [], [], [], [], [], [], [], [], []]

In [41]:
chart[1]

'Bad Blood' by Taylor Swift Featuring Kendrick Lamar

In [42]:
songid = chart[1].spotifyID
response = requests.get('https://api.spotify.com/v1/tracks/' + songid)

In [43]:
data = response.json()

In [44]:
albumid = data['album']['id']
response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
data = response.json()

In [48]:
data['genres']

[]

Okay, it's not looking good.

Liner notes and Google Music also aren't looking the greatest, although maybe I just don't know how to look at Google Music's data.
<p>I'm going to just try looking at all of the genres that artist is associated with.

In [51]:
chart[0]

'See You Again' by Wiz Khalifa Featuring Charlie Puth

In [52]:
chart[0].artist

'Wiz Khalifa Featuring Charlie Puth'

... going to deal with the "featuring" later if I get this to work.

In [53]:
chart[1].artist

'Taylor Swift Featuring Kendrick Lamar'

In [54]:
chart[2].artist

'Fetty Wap'

In [76]:
chart[3].artist

'OMI'

In [65]:
songid = chart[2].spotifyID
response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
data = response.json()

In [73]:
data['artists'][0]['id']

'6PXS4YHDkKvl1wkIl4V8DL'

In [74]:
artistid = data['artists'][0]['id']
response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
data = response.json()

In [75]:
data

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6PXS4YHDkKvl1wkIl4V8DL'},
 'followers': {'href': None, 'total': 1479193},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/6PXS4YHDkKvl1wkIl4V8DL',
 'id': '6PXS4YHDkKvl1wkIl4V8DL',
 'images': [{'height': 563,
   'url': 'https://i.scdn.co/image/0d56632017e9a02e8a5d5f8f7babf002f019a27b',
   'width': 1000},
  {'height': 360,
   'url': 'https://i.scdn.co/image/4482faece7e5973d51b458b6d46823c15a43127e',
   'width': 640},
  {'height': 113,
   'url': 'https://i.scdn.co/image/4e1c91b398028a44ed7ade9dca6fbad4cf2ebb63',
   'width': 200},
  {'height': 36,
   'url': 'https://i.scdn.co/image/c9da9c8aeee9487838cd997bbc264266ae3d8123',
   'width': 64}],
 'name': 'Fetty Wap',
 'popularity': 87,
 'type': 'artist',
 'uri': 'spotify:artist:6PXS4YHDkKvl1wkIl4V8DL'}

In [77]:
songid = chart[3].spotifyID
response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
data = response.json()
artistid = data['artists'][0]['id']
response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
data = response.json()

In [78]:
data

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5MouCg6ta7zAxsfMEbc1uh'},
 'followers': {'href': None, 'total': 281098},
 'genres': ['dark hardcore'],
 'href': 'https://api.spotify.com/v1/artists/5MouCg6ta7zAxsfMEbc1uh',
 'id': '5MouCg6ta7zAxsfMEbc1uh',
 'images': [{'height': 686,
   'url': 'https://i.scdn.co/image/c8f8dbefcec3a9caf5f458ead8ef5ea16962f26a',
   'width': 1000},
  {'height': 439,
   'url': 'https://i.scdn.co/image/d32d6d23df327b66c6c0c2350d36d5c184681149',
   'width': 640},
  {'height': 137,
   'url': 'https://i.scdn.co/image/64ef86aa48ff09d4108748798640cb2716858a96',
   'width': 200},
  {'height': 44,
   'url': 'https://i.scdn.co/image/cf6473a38311a92bda898898c4ddbb0e3450f4e2',
   'width': 64}],
 'name': 'OMI',
 'popularity': 73,
 'type': 'artist',
 'uri': 'spotify:artist:5MouCg6ta7zAxsfMEbc1uh'}

### MY FIRST SUCCESS! on the fourth song...

In [79]:
songid = chart[0].spotifyID
response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
data = response.json()
artistid = data['artists'][0]['id']
response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
data = response.json()

In [85]:
data['genres']

['shimmer pop']

### OKAY, going for it: The title of this list is GENRES and it's for the data 7/01/2015:

In [86]:
genres = []
for item in chart[:5]:
    songid = item.spotifyID
    response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
    data = response.json()
    artistid = data['artists'][0]['id']
    response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
    data = response.json()
    genres.append(data['genres'])

In [87]:
genres

[['alternative hip hop', 'pop rap'],
 [],
 [],
 ['dark hardcore'],
 ['shimmer pop']]

In [88]:
for item in chart[5:15]:
    songid = item.spotifyID
    response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
    data = response.json()
    artistid = data['artists'][0]['id']
    response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
    data = response.json()
    genres.append(data['genres'])

In [89]:
genres

[['alternative hip hop', 'pop rap'],
 [],
 [],
 ['dark hardcore'],
 ['shimmer pop'],
 [],
 [],
 ['edm', 'electro house', 'house'],
 [],
 ['indie r&b'],
 [],
 ['indie r&b'],
 [],
 [],
 []]

In [90]:
for item in chart[15:50]:
    songid = item.spotifyID
    response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
    data = response.json()
    artistid = data['artists'][0]['id']
    response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
    data = response.json()
    genres.append(data['genres'])

KeyError: 'artists'

In [91]:
genres

[['alternative hip hop', 'pop rap'],
 [],
 [],
 ['dark hardcore'],
 ['shimmer pop'],
 [],
 [],
 ['edm', 'electro house', 'house'],
 [],
 ['indie r&b'],
 [],
 ['indie r&b'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['pop', 'pop christmas', 'pop rock'],
 [],
 [],
 ['indie r&b'],
 [],
 ['metropopolis', 'pop', 'synthpop'],
 ['pop', 'pop christmas', 'pop rock'],
 [],
 ['contemporary country'],
 ['dance pop', 'pop', 'pop christmas'],
 [],
 [],
 ['hip pop'],
 [],
 [],
 [],
 [],
 ['contemporary country', 'country road'],
 []]

14 with genres,
<p>23 without....
<p>Doesn't seem good enough.

Okay... let me see whether the top songs are also on the Billboard genre lists. There is:
* The Hot 100
* Hot Country Songs 'country-songs'
* Hot Rock Songs 'rock-songs'
* Alternative Songs 'alternative-songs'
* Hot R&B/Hip-Hop Songs 'r-b-hip-hop-songs'
* Hot Rap Songs 'rap-song'
* Hot Dance/Electronic Songs 'dance-electronic-songs'
* Hot Latin Songs 'latin-songs'
* Hot Christian Songs 'christian-songs'
* Hot Gospel Songs 'gospel-songs'
* Hot Holiday Songs 'hot-holiday-songs'

In [119]:
hot100chart = billboard.ChartData('hot-100')

In [115]:
genres = ['country-songs', 'rock-songs', 'alternative-songs', 'r-b-hip-hop-songs', 'rap-song', 'dance-electronic-songs', 'latin-songs', 'christian-songs', 'gospel-songs', 'hot-holiday-songs']
list_of_dicts = []
for genre in genres:
    dicts = {}
    chart = billboard.ChartData(genre)
    dicts['genre'] = genre
    dicts['spotIDs'] = [item.spotifyID for item in chart]
    list_of_dicts.append(dicts)

In [160]:
list_of_dicts

[{'genre': 'country-songs',
  'spotIDs': ['19PJf5HzeiVW0mdk82Swmn',
   '63lHuquEopEUiwPy6VLxlb',
   '0AdjJ2SaOeb5bPJ67nDbsW',
   '67citk3uzWs5qbaIVKTeg8',
   '0ohEiGlkEnNCJP1cr7xK08',
   '1R9FxlRkvKtt36dtTHs93X',
   '596ShP1KQsIzk7dTpJNXJi',
   '2wqXQjOmQ9KoBm9Ssgt4Fo',
   '5BbYPT6ehZf0hSv2su6JAl',
   '7dPEZhhG9RD6FwRHpqIJPw',
   '0iIReFSw0XoZ61SDy5uaqU',
   '3eGH1kOyOjq19khdfYg5R7',
   '3JffB4CABj9lA0NC63kbCp',
   '5BIwPoVMFnFIr8RzJunRvf',
   '1lGOow8CQ1sk4NDHJgXXUJ',
   '',
   '1HwAeValR9dJzAf8Erlv3s',
   '5PNcJn4oFNvlRfrZBHfqWh',
   '38gZVjeaeR3gcnCzziAxBm',
   '3rN1zT8D6GX61NvQJ37UQc',
   '4DdF1l0DzBQy8YL2UVhPrU',
   '2ULh6nNdjEs8kQB6Ry0HHY',
   '4plhHbrWLbmIDCLiqesqdq',
   '7DYswEnZFL6SKaNLTxBf5b',
   '1xT8auZdc8Rksi7RSGZoax']},
 {'genre': 'rock-songs',
  'spotIDs': ['2Z8WuEywRWYTKe1NybPQEW',
   '6i0V12jOa3mr6uu4WYhUBr',
   '3CRDbSIZ4r5MsZ0YwxuEkn',
   '2ggSyGB5HnVvGDGofu3ITZ',
   '3RiPr603aXAoi4GHyXx0uy',
   '7lGKEWMXVWWTt3X71Bv44I',
   '1Cj2vqUwlJVG27gJrun92y',
   '4dASQiO1Eoo3R

In [161]:
num = 0
for item in hot100chart:
#     print(item)
    num += 1
    print(num)
    print(item)
    print(item.spotifyID)
    for genre_dict in list_of_dicts:
        if item.spotifyID in genre_dict['spotIDs']:
            print(genre_dict['genre'])
    print('')

1
'Cheap Thrills' by Sia Featuring Sean Paul
7BPw9h8vCIX94N0rjFU12S

2
'One Dance' by Drake Featuring WizKid & Kyla
11hqMWwX7sF3sOGdtijofF
r-b-hip-hop-songs

3
'This Is What You Came For' by Calvin Harris Featuring Rihanna
0azC730Exh71aQlOt9Zj3y
dance-electronic-songs

4
'Can't Stop The Feeling!' by Justin Timberlake
6JV2JOEocMgcZxYSZelKcc

5
'Don't Let Me Down' by The Chainsmokers Featuring Daya
1i1fxkWeaMmKEB4T7zqbzK
dance-electronic-songs

6
'Ride' by twenty one pilots
2Z8WuEywRWYTKe1NybPQEW
rock-songs
alternative-songs

7
'Needed Me' by Rihanna
3kfxarilcBr81mb2hmZLeh
r-b-hip-hop-songs

8
'Don't Mind' by Kent Jones
5c6vPEx6QTUTjmLiza1bpy
r-b-hip-hop-songs
rap-song

9
'Panda' by Desiigner
6r2jK1A6oFRPREZfxjc5d1
r-b-hip-hop-songs
rap-song

10
'Send My Love (To Your New Lover)' by Adele
70uEe3Key95fjdHfcf5ELt

11
'Rise' by Katy Perry
1CwNogTShsnsn1C8UhRmYX

12
'Just Like Fire' by P!nk
7K5dzhGda2vRTaAWYI3hrb

13
'Work From Home' by Fifth Harmony Featuring Ty Dolla $ign
0SJPThTy7ynySPF4e

Okay, now there's more genres.

Well, let's see how far Billboard goes back.

In [122]:
chart1980 = billboard.ChartData('hot-100', date = '1980-07-01')

In [123]:
chart1980[0]

'Coming Up (Live At Glasgow)' by Paul McCartney & Wings

In [129]:
chart1960 = billboard.ChartData('hot-100', date = '1960-07-01')

In [130]:
chart1960

hot-100 chart from 1960-07-02
-----------------------------
1. 'Everybody's Somebody's Fool' by Connie Francis (+1)
2. 'Cathy's Clown' by The Everly Brothers (-1)
3. 'Alley-Oop' by Hollywood Argyles (+3)
4. 'Burning Bridges' by Jack Scott (-1)
5. 'Because They're Young' by Duane Eddy And The Rebels (+6)
6. 'I'm Sorry' by Brenda Lee (+8)
7. 'A Rockin' Good Way (To Mess Around And Fall In Love)' by Dinah Washington & Brook Benton (+6)
8. 'Paper Roses' by Anita Bryant (-1)
9. 'Good Timin'' by Jimmy Jones (-5)
10. 'Swingin' School' by Bobby Rydell (-5)
11. 'My Home Town' by Paul Anka (+1)
12. 'Wonderful World' by Sam Cooke (+3)
13. 'Love You So' by Ron Holden with The Thunderbirds (-5)
14. 'Happy-Go-Lucky-Me' by Paul Evans (+2)
15. 'Mule Skinner Blues' by The Fendermen (+6)
16. 'Alley-Oop' by Dante and the Evergreens (+4)
17. 'That's All You Gotta Do' by Brenda Lee (+19)
18. 'Young Emotions' by Ricky Nelson (0)
19. 'He'll Have To Stay' by Jeanne Black (-10)
20. 'Jealous Of You (Tango Della

Change in plans. I'm going to see how many songs are cross-genre according to Billboard.
<p>First, let's get the data for every week.

In [153]:
import calendar
import datetime

I'm choosing Wednesdays because Billboard updates weekly and I'm guessing the day of the week wouldn't be a Wednesday.

In [148]:
cal = calendar.Calendar(2)

In [149]:
cal

In [157]:
wednesdays2016 = []
for month in range(1, 13):
    for item in cal.monthdatescalendar(2016, month):
        wednesdays2016.append(datetime.datetime.strftime(item[0], '%Y-%m-%d'))

In [158]:
wednesdays2016

['2015-12-30',
 '2016-01-06',
 '2016-01-13',
 '2016-01-20',
 '2016-01-27',
 '2016-01-27',
 '2016-02-03',
 '2016-02-10',
 '2016-02-17',
 '2016-02-24',
 '2016-02-24',
 '2016-03-02',
 '2016-03-09',
 '2016-03-16',
 '2016-03-23',
 '2016-03-30',
 '2016-03-30',
 '2016-04-06',
 '2016-04-13',
 '2016-04-20',
 '2016-04-27',
 '2016-04-27',
 '2016-05-04',
 '2016-05-11',
 '2016-05-18',
 '2016-05-25',
 '2016-06-01',
 '2016-06-08',
 '2016-06-15',
 '2016-06-22',
 '2016-06-29',
 '2016-06-29',
 '2016-07-06',
 '2016-07-13',
 '2016-07-20',
 '2016-07-27',
 '2016-07-27',
 '2016-08-03',
 '2016-08-10',
 '2016-08-17',
 '2016-08-24',
 '2016-08-31',
 '2016-08-31',
 '2016-09-07',
 '2016-09-14',
 '2016-09-21',
 '2016-09-28',
 '2016-09-28',
 '2016-10-05',
 '2016-10-12',
 '2016-10-19',
 '2016-10-26',
 '2016-10-26',
 '2016-11-02',
 '2016-11-09',
 '2016-11-16',
 '2016-11-23',
 '2016-11-30',
 '2016-11-30',
 '2016-12-07',
 '2016-12-14',
 '2016-12-21',
 '2016-12-28']

Eventually I want a table:
    `[
    {
    'year': 2016, 
    'week': '2016-12-28', 
    'pos': 1, 
    'chartobject': chartobject, 
    'artist': 'Sia' ## GET RID OF THE FEATURING...
    'track': 'Going Crazy'
    'spotID': '5BbYPT6ehZf0hSv2su6JAl',
    'any_genre': True, 
    'multi-genre': True, 
    'main_genre': '', 
    'all_genres': ['pop', 'reggae', 'indie'], 
    'source_of_genres': 'Billboard lists'},
    {...},
    ...
    ]`

Oh no. Found a bug in my code. Because I was matching up the Spotify ID between all of the Billboard charts, any song without a Spotify ID was matching up with all of the other songs without Spotify IDs... So all of the weird cross-genre songs were not really cross-genre.

In [162]:
hot100s = []
temporarydict = {}    

In [163]:
chart2015_12_30 = billboard.ChartData('hot-100', date = wednesdays2016[0])

In [164]:
genres = ['country-songs', 'rock-songs', 'alternative-songs', 'r-b-hip-hop-songs', 'rap-song', 'dance-electronic-songs', 'latin-songs', 'christian-songs', 'gospel-songs', 'hot-holiday-songs']
hotgenre100s = []
for genre in genres:
    dicts = {}
    chart = billboard.ChartData(genre)
    dicts['genre'] = genre
    dicts['spotIDs'] = [item.spotifyID for item in chart]
    hotgenre100s.append(dicts)

In [168]:
hotgenre100s

[{'genre': 'country-songs',
  'spotIDs': ['19PJf5HzeiVW0mdk82Swmn',
   '63lHuquEopEUiwPy6VLxlb',
   '0AdjJ2SaOeb5bPJ67nDbsW',
   '67citk3uzWs5qbaIVKTeg8',
   '0ohEiGlkEnNCJP1cr7xK08',
   '1R9FxlRkvKtt36dtTHs93X',
   '596ShP1KQsIzk7dTpJNXJi',
   '2wqXQjOmQ9KoBm9Ssgt4Fo',
   '5BbYPT6ehZf0hSv2su6JAl',
   '7dPEZhhG9RD6FwRHpqIJPw',
   '0iIReFSw0XoZ61SDy5uaqU',
   '3eGH1kOyOjq19khdfYg5R7',
   '3JffB4CABj9lA0NC63kbCp',
   '5BIwPoVMFnFIr8RzJunRvf',
   '1lGOow8CQ1sk4NDHJgXXUJ',
   '',
   '1HwAeValR9dJzAf8Erlv3s',
   '5PNcJn4oFNvlRfrZBHfqWh',
   '38gZVjeaeR3gcnCzziAxBm',
   '3rN1zT8D6GX61NvQJ37UQc',
   '4DdF1l0DzBQy8YL2UVhPrU',
   '2ULh6nNdjEs8kQB6Ry0HHY',
   '4plhHbrWLbmIDCLiqesqdq',
   '7DYswEnZFL6SKaNLTxBf5b',
   '1xT8auZdc8Rksi7RSGZoax']},
 {'genre': 'rock-songs',
  'spotIDs': ['2Z8WuEywRWYTKe1NybPQEW',
   '6i0V12jOa3mr6uu4WYhUBr',
   '3CRDbSIZ4r5MsZ0YwxuEkn',
   '2ggSyGB5HnVvGDGofu3ITZ',
   '3RiPr603aXAoi4GHyXx0uy',
   '7lGKEWMXVWWTt3X71Bv44I',
   '1Cj2vqUwlJVG27gJrun92y',
   '4dASQiO1Eoo3R

## Genres from Billboard

In [170]:
num = 0
for item in chart2015_12_30:
#     print(item)
    num += 1
    print(num)
    print(item)
    print(item.spotifyID)
    if item.spotifyID:
        for genre_dict in hotgenre100s:
            if item.spotifyID in genre_dict['spotIDs']:
                print('FROM BILLBOARD:', genre_dict['genre'])
    print('')

1
'Hello' by Adele
1MDoll6jK4rrk2BcFRP5i7

2
'Sorry' by Justin Bieber
2GpHJb7xGNBaIDTrBxjIu6

3
'Hotline Bling' by Drake
1UfBAJfmofTffrae5ls6DA

4
'Love Yourself' by Justin Bieber
3hB5DgAiMAQ4DzYbsMq1IT

5
'What Do You Mean?' by Justin Bieber
1ds2QsfhAAfRiaFMGDzrdb

6
'Same Old Love' by Selena Gomez
7jv4Mb21bdd9SDVOm9Fm9l

7
'The Hills' by The Weeknd
0ysmhSqZrkiRkRH4mH8rDn

8
'Here' by Alessia Cara
0yKXxMgfRpYzVeIbPZss5B

9
'Stitches' by Shawn Mendes
3zkWCteF82vJwv0hRLba76

10
'Like I'm Gonna Lose You' by Meghan Trainor Featuring John Legend
2YlZnw2ikdb837oKMKjBkW

11
'679' by Fetty Wap Featuring Remy Boyz
5NQbUaeTEOGdD6hHcre0dZ

12
'In The Night' by The Weeknd
7fPHfBCyKE3aVCBjE4DAvl

13
'Jumpman' by Drake & Future


14
'On My Mind' by Ellie Goulding
0kN5Ohxu97lwdDmP5n7FZE

15
'Can't Feel My Face' by The Weeknd
3X38ErFiKgzUxinBlhwuWm

16
'Antidote' by Travi$ Scott
0KaA33vmmUEY77YD6L3KFi

17
'Ex's & Oh's' by Elle King
70eDxAyAraNTiD6lx2ZEnH

18
'All I Want For Christmas Is You' by Maria

## Genres from Spotify

In [171]:
for item in chart2015_12_30[:20]:
    songid = item.spotifyID
    response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
    data = response.json()
    albumid = data['album']['id']
    print('albumid:', albumid)
    response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
    data = response.json()
    print(data['genres'])

#     genres.append(data['genres'])

albumid: 5y28RdPzIRXegzr5YM9Y0D
[]
albumid: 7sIgLnbKhrEN5k0tbB7xbI
[]
albumid: 2rTxwqA6v4lccbPKD31CQI
[]
albumid: 7fZH0aUAjY3ay25obOUf2a
[]
albumid: 2hL8vuRtlo75Wr9PyZI5Jb
[]
albumid: 5k70b7FbvYTLr31lFyYhJI
[]
albumid: 1XO0SIspYtH1EKFs17UAtj
[]
albumid: 0yw6xPSj5WHurxd6l4VjtG
[]
albumid: 0CnQ0JQajNswRjPkNYVG8m
[]
albumid: 5W98Ab4VvQEuFEE4TIe5fE
[]
albumid: 0TyDTzG2zt2tg0wONT9wSU
[]
albumid: 28ZKQMoNBB0etKXZ97G2SN
[]


KeyError: 'album'

## Genres from last.fm

In [173]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [175]:
html_str = urlopen('http://www.last.fm/music/Taylor+Swift/_/Bad+Blood').read()

In [176]:
document = BeautifulSoup(html_str, 'html.parser')

In [177]:
first_tag = document.find('li', {'class': 'tag'})

In [178]:
first_tag

<li class="tag">
<a href="/tag/pop">pop</a></li>

In [187]:
lastfmgenre = first_tag.find('a').string

In [188]:
lastfmgenre

'pop'

### Trying last.fm with a chart item:

In [191]:
chartitem = chart2015_12_30[0]
chartitem

'Hello' by Adele

In [192]:
artist = chartitem.artist.replace(' ', '+')
artist

'Adele'

In [195]:
track = chartitem.title.replace(' ', '+')
track

'Hello'

In [196]:
html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()

In [197]:
document = BeautifulSoup(html_str, 'html.parser')
first_tag = document.find('li', {'class': 'tag'})
lastfmgenre = first_tag.find('a').string

In [198]:
lastfmgenre

'soul'

## The Table Again
    `hot100s = [
    {
    'year': 2016, 
    'week': '2016-12-28', 
    'pos': 1, 
    'chartobject': chartobject, 
    'artist': 'Sia' ## GET RID OF THE FEATURING...
    'track': 'Going Crazy'
    'spotID': '5BbYPT6ehZf0hSv2su6JAl',
    'any_genre': True, 
    'multi-genre': True, 
    'main_genre': '', 
    'all_genres': ['pop', 'reggae', 'indie'], 
    'source_of_genres': 'Billboard lists'},
    {...},
    ...
    ]`

**The beginning of this list**

In [203]:
hot100s = []

In [206]:
for chartitem in chart2015_12_30[:1]:
    temporarydict = {}
    temporarydict['year'] = 2016 ## this will change as we feed the dates in
    temporarydict['week'] = '2016-12-28' ## this will also change
    temporarydict['pos'] = chartitem.rank
    temporarydict['chartobject'] = chartitem
    temporarydict['artist'] = chartitem.artist
    temporarydict['track'] = chartitem.title
    temporarydict['spotID'] = chartitem.spotifyID
    temporarydict['any_genre'] = False
#     temporarydict['multi-genre'] = False ## I might take this away
    temporarydict['main_genre'] = ''
#     temporarydict['all_genres'] = [] ## I might take this away
    temporarydict['source_of_genres'] = ''
    
    ## last.fm method
    lastfmgenre = None
    artist = chartitem.artist.replace(' ', '+')
    track = chartitem.title.replace(' ', '+')
    html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()
    document = BeautifulSoup(html_str, 'html.parser')
    first_tag = document.find('li', {'class': 'tag'})
    lastfmgenre = first_tag.find('a').string
    if lastfmgenre:
        temporarydict['any_genre'] = True
        temporarydict['main_genre'] = lastfmgenre
        temporarydict['source_of_genres'] = 'last.fm'
        
    ## end last.fm method
    hot100s.append(temporarydict)

artist: Adele
track: Hello
first_tag: <li class="tag">
<a href="/tag/soul">soul</a></li>
lastfmgenre: soul


In [209]:
hot100s

[{'any_genre': True,
  'artist': 'Adele',
  'chartobject': 'Hello' by Adele,
  'main_genre': 'soul',
  'pos': 1,
  'source_of_genres': 'last.fm',
  'spotID': '1MDoll6jK4rrk2BcFRP5i7',
  'track': 'Hello',
  'week': '2016-12-28',
  'year': 2016}]

In [211]:
hot100s

[{'any_genre': True,
  'artist': 'Adele',
  'chartobject': 'Hello' by Adele,
  'main_genre': 'soul',
  'pos': 1,
  'source_of_genres': 'last.fm',
  'spotID': '1MDoll6jK4rrk2BcFRP5i7',
  'track': 'Hello',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Justin Bieber',
  'chartobject': 'Sorry' by Justin Bieber,
  'main_genre': 'pop',
  'pos': 2,
  'source_of_genres': 'last.fm',
  'spotID': '2GpHJb7xGNBaIDTrBxjIu6',
  'track': 'Sorry',
  'week': '2016-12-28',
  'year': 2016}]

In [217]:
for chartitem in chart2015_12_30[2:3]:
    temporarydict = {}
    temporarydict['year'] = 2016 ## this will change as we feed the dates in
    temporarydict['week'] = '2016-12-28' ## this will also change
    temporarydict['pos'] = chartitem.rank
    temporarydict['chartobject'] = chartitem
    temporarydict['artist'] = chartitem.artist
    temporarydict['track'] = chartitem.title
    temporarydict['spotID'] = chartitem.spotifyID
    temporarydict['any_genre'] = False
#     temporarydict['multi-genre'] = False ## I might take this away
    temporarydict['main_genre'] = ''
#     temporarydict['all_genres'] = [] ## I might take this away
    temporarydict['source_of_genres'] = ''
    
    ## last.fm method
    artist = chartitem.artist.replace(' ', '+')
    track = chartitem.title.replace(' ', '+')
    html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()
    document = BeautifulSoup(html_str, 'html.parser')
    if document.find('li', {'class': 'tag'}):
        first_tag = document.find('li', {'class': 'tag'})
        lastfmgenre = first_tag.find('a').string
        if lastfmgenre:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = lastfmgenre
            temporarydict['source_of_genres'] = 'last.fm'
    ## end last.fm method
    
    ## spotify method
    else:
        songid = item.spotifyID
        response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
        data = response.json()
        albumid = data['album']['id']
        artistid = data['artists'][0]['id']
        response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
        data = response.json() 
        if data['genres']:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = data['genres']
            temporarydict['source_of_genres'] = 'spotify'
        else:
            response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
            data = response.json()
            if data['genres']:
                temporarydict['any_genre'] = True
                temporarydict['main_genre'] = data['genres'][0]
                temporarydict['source_of_genres'] = 'spotify'
    ## end spotify method
    
    hot100s.append(temporarydict)

In [220]:
hot100s

[{'any_genre': True,
  'artist': 'Adele',
  'chartobject': 'Hello' by Adele,
  'main_genre': 'soul',
  'pos': 1,
  'source_of_genres': 'last.fm',
  'spotID': '1MDoll6jK4rrk2BcFRP5i7',
  'track': 'Hello',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Justin Bieber',
  'chartobject': 'Sorry' by Justin Bieber,
  'main_genre': 'pop',
  'pos': 2,
  'source_of_genres': 'last.fm',
  'spotID': '2GpHJb7xGNBaIDTrBxjIu6',
  'track': 'Sorry',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Drake',
  'chartobject': 'Hotline Bling' by Drake,
  'main_genre': 'pop',
  'pos': 3,
  'source_of_genres': 'spotify',
  'spotID': '1UfBAJfmofTffrae5ls6DA',
  'track': 'Hotline Bling',
  'week': '2016-12-28',
  'year': 2016}]

In [221]:
for chartitem in chart2015_12_30[3:5]:
    temporarydict = {}
    temporarydict['year'] = 2016 ## this will change as we feed the dates in
    temporarydict['week'] = '2016-12-28' ## this will also change
    temporarydict['pos'] = chartitem.rank
    temporarydict['chartobject'] = chartitem
    temporarydict['artist'] = chartitem.artist
    temporarydict['track'] = chartitem.title
    temporarydict['spotID'] = chartitem.spotifyID
    temporarydict['any_genre'] = False
#     temporarydict['multi-genre'] = False ## I might take this away
    temporarydict['main_genre'] = ''
#     temporarydict['all_genres'] = [] ## I might take this away
    temporarydict['source_of_genres'] = ''
    
    ## last.fm method
    artist = chartitem.artist.replace(' ', '+')
    track = chartitem.title.replace(' ', '+')
    html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()
    document = BeautifulSoup(html_str, 'html.parser')
    if document.find('li', {'class': 'tag'}):
        first_tag = document.find('li', {'class': 'tag'})
        lastfmgenre = first_tag.find('a').string
        if lastfmgenre:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = lastfmgenre
            temporarydict['source_of_genres'] = 'last.fm'
    ## end last.fm method
    
    ## spotify method
    else:
        songid = item.spotifyID
        response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
        data = response.json()
        albumid = data['album']['id']
        artistid = data['artists'][0]['id']
        response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
        data = response.json() 
        if data['genres']:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = data['genres']
            temporarydict['source_of_genres'] = 'spotify'
        else:
            response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
            data = response.json()
            if data['genres']:
                temporarydict['any_genre'] = True
                temporarydict['main_genre'] = data['genres'][0]
                temporarydict['source_of_genres'] = 'spotify'
    ## end spotify method
    
    hot100s.append(temporarydict)

In [222]:
hot100s

[{'any_genre': True,
  'artist': 'Adele',
  'chartobject': 'Hello' by Adele,
  'main_genre': 'soul',
  'pos': 1,
  'source_of_genres': 'last.fm',
  'spotID': '1MDoll6jK4rrk2BcFRP5i7',
  'track': 'Hello',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Justin Bieber',
  'chartobject': 'Sorry' by Justin Bieber,
  'main_genre': 'pop',
  'pos': 2,
  'source_of_genres': 'last.fm',
  'spotID': '2GpHJb7xGNBaIDTrBxjIu6',
  'track': 'Sorry',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Drake',
  'chartobject': 'Hotline Bling' by Drake,
  'main_genre': 'pop',
  'pos': 3,
  'source_of_genres': 'spotify',
  'spotID': '1UfBAJfmofTffrae5ls6DA',
  'track': 'Hotline Bling',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Justin Bieber',
  'chartobject': 'Love Yourself' by Justin Bieber,
  'main_genre': 'pop',
  'pos': 4,
  'source_of_genres': 'spotify',
  'spotID': '3hB5DgAiMAQ4DzYbsMq1IT',
  'track': 'Love Yourself',


In [223]:
for chartitem in chart2015_12_30[5:8]:
    temporarydict = {}
    temporarydict['year'] = 2016 ## this will change as we feed the dates in
    temporarydict['week'] = '2016-12-28' ## this will also change
    temporarydict['pos'] = chartitem.rank
    temporarydict['chartobject'] = chartitem
    temporarydict['artist'] = chartitem.artist
    temporarydict['track'] = chartitem.title
    temporarydict['spotID'] = chartitem.spotifyID
    temporarydict['any_genre'] = False
#     temporarydict['multi-genre'] = False ## I might take this away
    temporarydict['main_genre'] = ''
#     temporarydict['all_genres'] = [] ## I might take this away
    temporarydict['source_of_genres'] = ''
    
    ## last.fm method
    artist = chartitem.artist.replace(' ', '+')
    track = chartitem.title.replace(' ', '+')
    html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()
    document = BeautifulSoup(html_str, 'html.parser')
    if document.find('li', {'class': 'tag'}):
        first_tag = document.find('li', {'class': 'tag'})
        lastfmgenre = first_tag.find('a').string
        if lastfmgenre:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = lastfmgenre
            temporarydict['source_of_genres'] = 'last.fm'
    ## end last.fm method
    
    ## spotify method
    else:
        songid = item.spotifyID
        response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
        data = response.json()
        albumid = data['album']['id']
        artistid = data['artists'][0]['id']
        response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
        data = response.json() 
        if data['genres']:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = data['genres']
            temporarydict['source_of_genres'] = 'spotify'
        else:
            response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
            data = response.json()
            if data['genres']:
                temporarydict['any_genre'] = True
                temporarydict['main_genre'] = data['genres'][0]
                temporarydict['source_of_genres'] = 'spotify'
    ## end spotify method
    
    hot100s.append(temporarydict)

In [224]:
hot100s

[{'any_genre': True,
  'artist': 'Adele',
  'chartobject': 'Hello' by Adele,
  'main_genre': 'soul',
  'pos': 1,
  'source_of_genres': 'last.fm',
  'spotID': '1MDoll6jK4rrk2BcFRP5i7',
  'track': 'Hello',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Justin Bieber',
  'chartobject': 'Sorry' by Justin Bieber,
  'main_genre': 'pop',
  'pos': 2,
  'source_of_genres': 'last.fm',
  'spotID': '2GpHJb7xGNBaIDTrBxjIu6',
  'track': 'Sorry',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Drake',
  'chartobject': 'Hotline Bling' by Drake,
  'main_genre': 'pop',
  'pos': 3,
  'source_of_genres': 'spotify',
  'spotID': '1UfBAJfmofTffrae5ls6DA',
  'track': 'Hotline Bling',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Justin Bieber',
  'chartobject': 'Love Yourself' by Justin Bieber,
  'main_genre': 'pop',
  'pos': 4,
  'source_of_genres': 'spotify',
  'spotID': '3hB5DgAiMAQ4DzYbsMq1IT',
  'track': 'Love Yourself',


In [225]:
for chartitem in chart2015_12_30[8:12]:
    temporarydict = {}
    temporarydict['year'] = 2016 ## this will change as we feed the dates in
    temporarydict['week'] = '2016-12-28' ## this will also change
    temporarydict['pos'] = chartitem.rank
    temporarydict['chartobject'] = chartitem
    temporarydict['artist'] = chartitem.artist
    temporarydict['track'] = chartitem.title
    temporarydict['spotID'] = chartitem.spotifyID
    temporarydict['any_genre'] = False
#     temporarydict['multi-genre'] = False ## I might take this away
    temporarydict['main_genre'] = ''
#     temporarydict['all_genres'] = [] ## I might take this away
    temporarydict['source_of_genres'] = ''
    
    ## last.fm method
    artist = chartitem.artist.replace(' ', '+')
    track = chartitem.title.replace(' ', '+')
    html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()
    document = BeautifulSoup(html_str, 'html.parser')
    if document.find('li', {'class': 'tag'}):
        first_tag = document.find('li', {'class': 'tag'})
        lastfmgenre = first_tag.find('a').string
        if lastfmgenre:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = lastfmgenre
            temporarydict['source_of_genres'] = 'last.fm'
    ## end last.fm method
    
    ## spotify method
    else:
        songid = item.spotifyID
        response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
        data = response.json()
        albumid = data['album']['id']
        artistid = data['artists'][0]['id']
        response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
        data = response.json() 
        if data['genres']:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = data['genres']
            temporarydict['source_of_genres'] = 'spotify'
        else:
            response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
            data = response.json()
            if data['genres']:
                temporarydict['any_genre'] = True
                temporarydict['main_genre'] = data['genres'][0]
                temporarydict['source_of_genres'] = 'spotify'
    ## end spotify method
    
    hot100s.append(temporarydict)

In [226]:
hot100s

[{'any_genre': True,
  'artist': 'Adele',
  'chartobject': 'Hello' by Adele,
  'main_genre': 'soul',
  'pos': 1,
  'source_of_genres': 'last.fm',
  'spotID': '1MDoll6jK4rrk2BcFRP5i7',
  'track': 'Hello',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Justin Bieber',
  'chartobject': 'Sorry' by Justin Bieber,
  'main_genre': 'pop',
  'pos': 2,
  'source_of_genres': 'last.fm',
  'spotID': '2GpHJb7xGNBaIDTrBxjIu6',
  'track': 'Sorry',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Drake',
  'chartobject': 'Hotline Bling' by Drake,
  'main_genre': 'pop',
  'pos': 3,
  'source_of_genres': 'spotify',
  'spotID': '1UfBAJfmofTffrae5ls6DA',
  'track': 'Hotline Bling',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Justin Bieber',
  'chartobject': 'Love Yourself' by Justin Bieber,
  'main_genre': 'pop',
  'pos': 4,
  'source_of_genres': 'spotify',
  'spotID': '3hB5DgAiMAQ4DzYbsMq1IT',
  'track': 'Love Yourself',


In [227]:
for chartitem in chart2015_12_30[12:20]:
    temporarydict = {}
    temporarydict['year'] = 2015 ## this will change as we feed the dates in
    temporarydict['week'] = '2015-12-30' ## this will also change
    temporarydict['pos'] = chartitem.rank
    temporarydict['chartobject'] = chartitem
    temporarydict['artist'] = chartitem.artist
    temporarydict['track'] = chartitem.title
    temporarydict['spotID'] = chartitem.spotifyID
    temporarydict['any_genre'] = False
#     temporarydict['multi-genre'] = False ## I might take this away
    temporarydict['main_genre'] = ''
#     temporarydict['all_genres'] = [] ## I might take this away
    temporarydict['source_of_genres'] = ''
    
    ## last.fm method
    artist = chartitem.artist.replace(' ', '+')
    track = chartitem.title.replace(' ', '+')
    html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()
    document = BeautifulSoup(html_str, 'html.parser')
    if document.find('li', {'class': 'tag'}):
        first_tag = document.find('li', {'class': 'tag'})
        lastfmgenre = first_tag.find('a').string
        if lastfmgenre:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = lastfmgenre
            temporarydict['source_of_genres'] = 'last.fm'
    ## end last.fm method
    
    ## spotify method
    else:
        songid = item.spotifyID
        response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
        data = response.json()
        albumid = data['album']['id']
        artistid = data['artists'][0]['id']
        response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
        data = response.json() 
        if data['genres']:
            temporarydict['any_genre'] = True
            temporarydict['main_genre'] = data['genres']
            temporarydict['source_of_genres'] = 'spotify'
        else:
            response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
            data = response.json()
            if data['genres']:
                temporarydict['any_genre'] = True
                temporarydict['main_genre'] = data['genres'][0]
                temporarydict['source_of_genres'] = 'spotify'
    ## end spotify method
    
    hot100s.append(temporarydict)

In [228]:
hot100s[-8:]

[{'any_genre': True,
  'artist': 'Drake & Future',
  'chartobject': 'Jumpman' by Drake & Future,
  'main_genre': 'pop',
  'pos': 13,
  'source_of_genres': 'spotify',
  'spotID': '',
  'track': 'Jumpman',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Ellie Goulding',
  'chartobject': 'On My Mind' by Ellie Goulding,
  'main_genre': 'pop',
  'pos': 14,
  'source_of_genres': 'spotify',
  'spotID': '0kN5Ohxu97lwdDmP5n7FZE',
  'track': 'On My Mind',
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'The Weeknd',
  'chartobject': 'Can't Feel My Face' by The Weeknd,
  'main_genre': 'love at first listen',
  'pos': 15,
  'source_of_genres': 'last.fm',
  'spotID': '3X38ErFiKgzUxinBlhwuWm',
  'track': "Can't Feel My Face",
  'week': '2016-12-28',
  'year': 2016},
 {'any_genre': True,
  'artist': 'Travi$ Scott',
  'chartobject': 'Antidote' by Travi$ Scott,
  'main_genre': 'hip-hop',
  'pos': 16,
  'source_of_genres': 'last.fm',
  'spotID': '0KaA

Let's save the work as a CSV

In [2]:
import pandas as pd

Bah! I messed up the year and week.

In [238]:
for item in hot100s:
    item['year'] = 2015 ## this will change as we feed the dates in
    item['week'] = '2015-12-30' ## this will also change

In [239]:
hot100s[0]

{'any_genre': True,
 'artist': 'Adele',
 'chartobject': 'Hello' by Adele,
 'main_genre': 'soul',
 'pos': 1,
 'source_of_genres': 'last.fm',
 'spotID': '1MDoll6jK4rrk2BcFRP5i7',
 'track': 'Hello',
 'week': '2015-12-30',
 'year': 2015}

In [240]:
df_20_chart2015_12_30 = pd.DataFrame(hot100s)
df_20_chart2015_12_30.to_csv('df_20_chart2015_12_30.csv')

**wednesdays2016[1]**

In [234]:
wednesdays2016[1]

'2016-01-06'

In [235]:
chart2016_01_06 = billboard.ChartData('hot-100', date = wednesdays2016[1])

In [270]:
for chartitem in chart2016_01_06[2:4]: ## this will also change
    temporarydict = {}
    temporarydict['year'] = 2016 ## this will change as we feed the dates in
    temporarydict['week'] = wednesdays2016[1] ## this will also change
    temporarydict['pos'] = chartitem.rank
    temporarydict['artist'] = chartitem.artist
    temporarydict['track'] = chartitem.title
    if chartitem.spotifyID:
        temporarydict['spotID'] = chartitem.spotifyID
        temporarydict['any_genre'] = False
    #     temporarydict['multi-genre'] = False ## I might take this away
        temporarydict['main_genre'] = ''
    #     temporarydict['all_genres'] = [] ## I might take this away
        temporarydict['source_of_genres'] = ''

        ## check past 2 rankings
        for d in hot100s[-20:]:
            if chartitem.spotifyID == d['spotID']:
                temporarydict['any_genre'] = True
                temporarydict['main_genre'] = d['main_genre']
                temporarydict['source_of_genres'] = d['source_of_genres']  
        ## check past 2 rankings

        ## last.fm method
        if temporarydict['any_genre'] == False:
            artist = chartitem.artist.replace(' ', '+')
            track = chartitem.title.replace(' ', '+')
            html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()
            document = BeautifulSoup(html_str, 'html.parser')
            if document.find('li', {'class': 'tag'}):
                first_tag = document.find('li', {'class': 'tag'})
                lastfmgenre = first_tag.find('a').string
                if lastfmgenre:
                    temporarydict['any_genre'] = True
                    temporarydict['main_genre'] = lastfmgenre
                    temporarydict['source_of_genres'] = 'last.fm'
        ## end last.fm method

        ## spotify method
        if temporarydict['any_genre'] == False:
            songid = chartitem.spotifyID
            response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
            data = response.json()
            albumid = data['album']['id']
            artistid = data['artists'][0]['id']
            response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
            data = response.json() 
            if data['genres']:
                temporarydict['any_genre'] = True
                temporarydict['main_genre'] = data['genres']
                temporarydict['source_of_genres'] = 'spotify'
            else:
                response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
                data = response.json()
                if data['genres']:
                    temporarydict['any_genre'] = True
                    temporarydict['main_genre'] = data['genres'][0]
                    temporarydict['source_of_genres'] = 'spotify'
        ## end spotify method
        
        ## hand-check
        if temporarydict['any_genre'] == False:
                    print(temporarydict)
                    print('')
    else: 
        print(temporarydict)
        print('')
        ## end hand-check
    
    
    hot100s.append(temporarydict)

# FUNCTIONS

In [368]:
hot100s

[{'artist': 'Adele',
  'genres': [],
  'pos': 1,
  'source_of_genres': '',
  'spotID': '1MDoll6jK4rrk2BcFRP5i7',
  'track': 'Hello',
  'week': '2015-12-30',
  'year': '2015'},
 {'artist': 'Justin Bieber',
  'genres': [],
  'pos': 2,
  'source_of_genres': '',
  'spotID': '2GpHJb7xGNBaIDTrBxjIu6',
  'track': 'Sorry',
  'week': '2015-12-30',
  'year': '2015'}]

In [4]:
import time
import re
import pandas as pd
# def def_recent_rankings():
#     if len(hot100s) < 40:
#         return hot100s
#     else: 
#         return hot100s[-40:]
def check_recent_rankings(): 
    for d in recent_rankings:
        try:
            if chartitem.spotifyID == d['spotID']:
                temporarydict['genres'] = d['genres']
                temporarydict['source_of_genres'] = d['source_of_genres']
        except:
            pass 
def check_recent_rankings_sans_spotID():
    for d in recent_rankings:
        try:
            if chartitem.title == d['track']:
                temporarydict['genres'] = d['genres']
                temporarydict['source_of_genres'] = d['source_of_genres']
        except:
            pass 
def last_fm_method():
    artist = re.sub(r' Featuring.+$', '', chartitem.artist).replace(' ', '+')
    track = chartitem.title.replace(' ', '+')
    try:
        html_str = urlopen('http://www.last.fm/music/' + artist + '/_/' + track).read()
        document = BeautifulSoup(html_str, 'html.parser')
        if document.find('li', {'class': 'tag'}):
            first_tag = document.find('li', {'class': 'tag'})
            temporarydict['genres'] = [first_tag.find('a').string] #only taking one bc some of the genres aren't really genres
            temporarydict['source_of_genres'] = 'last.fm'
    except:
        pass
    time.sleep(.3)
def spotify_method():
    songid = chartitem.spotifyID
    response = requests.get('https://api.spotify.com/v1/tracks/' + songid)
    data = response.json()
    albumid = data['album']['id']
    artistid = data['artists'][0]['id']
    response = requests.get('https://api.spotify.com/v1/albums/' + albumid)
    data = response.json() 
    if data['genres']:
        temporarydict['source_of_genres'] = 'spotify'
        temporarydict['genres'] = [data['genres']]
    else:
        response = requests.get('https://api.spotify.com/v1/artists/' + artistid)
        data = response.json()
        if data['genres']:
            temporarydict['source_of_genres'] = 'spotify'
            temporarydict['genres'] = data['genres']
def build_billboard_genre_charts(chartdate):
    bbgenres = ['pop-songs', 'country-songs', 'rock-songs', 'alternative-songs', 'r-b-hip-hop-songs', 'rap-song', 'dance-electronic-songs', 'latin-songs', 'christian-songs', 'gospel-songs', 'hot-holiday-songs']
    hotgenre100s = []
    for bbgenre in bbgenres:
        dicts = {}
        bbchart = billboard.ChartData(bbgenre, date = chartdate)
        dicts['genre'] = bbgenre
        dicts['artist'] = [item.artist for item in bbchart] # based on the fact that an artist is not likely to have multiple albums on the billboard lists at the same time
        hotgenre100s.append(dicts)
    return hotgenre100s
def billboard_method():    
    bbgenres = []
    if chartitem.artist: 
        for genre_dict in hotgenre100s:
            if chartitem.artist in genre_dict['artist']:
                bbgenres.append(genre_dict['genre'])
    if len(bbgenres) > 0:
        temporarydict['source_of_genres'] = 'billboard'
        temporarydict['genres'] = bbgenres

In [9]:
# hot100s = [] ## eventually move this
for x in range(967, 3498):
    chartdate = wednesdays[x] ## this changes

    try:
        chart = billboard.ChartData('hot-100', date = chartdate)
        hotgenres100s = None
        recent_rankings = hot100s[-40:]

        for chartitem in chart[:20]:
            temporarydict = {}
            temporarydict['year'] = chartdate[:4]
            temporarydict['week'] = chartdate
            temporarydict['pos'] = chartitem.rank
            temporarydict['artist'] = chartitem.artist
            temporarydict['track'] = chartitem.title
            temporarydict['genres'] = []
            temporarydict['source_of_genres'] = ''

            if chartitem.spotifyID:
                temporarydict['spotID'] = chartitem.spotifyID
                check_recent_rankings()
                if len(temporarydict['genres']) == 0:
                    last_fm_method()
                if len(temporarydict['genres']) == 0:
                    spotify_method()
                if len(temporarydict['genres']) == 0:
                    if hotgenres100s == None:
                        hotgenre100s = build_billboard_genre_charts(chartdate)
                    billboard_method()

            else:
                check_recent_rankings_sans_spotID()
                if len(temporarydict['genres']) == 0:
                    last_fm_method()
                if len(temporarydict['genres']) == 0:
                    if hotgenres100s == None:
                        hotgenre100s = build_billboard_genre_charts(chartdate)
                    billboard_method()
            ## hand-check
            if len(temporarydict['genres']) == 0:
                        print(temporarydict)
                        print('')

            hot100s.append(temporarydict)
#             time.sleep(1)
        pd.DataFrame(hot100s).to_csv('df_20_chart'+chartdate+'.csv', index=False)
    except:
        print('Could not pull data for:')
        print('    ', x)
        print('    ', chartdate)

{'year': '1975', 'week': '1975-06-25', 'source_of_genres': '', 'track': 'Love Will Keep Us Together', 'genres': [], 'artist': 'Captain & Tennille', 'spotID': '56ZPl9gFBZxPfRBOBqHfer', 'pos': 1}

{'year': '1975', 'week': '1975-06-25', 'source_of_genres': '', 'track': 'Wildfire', 'genres': [], 'artist': 'Michael Murphey', 'spotID': '4rHOEGE0pH206Q987k7MqF', 'pos': 3}

{'year': '1975', 'week': '1975-06-25', 'source_of_genres': '', 'track': "I'm Not Lisa", 'genres': [], 'artist': 'Jessi Colter', 'spotID': '2G9HPwBaHica41sMtQ7QvX', 'pos': 4}

{'year': '1975', 'week': '1975-06-25', 'source_of_genres': '', 'track': 'The Hustle', 'genres': [], 'artist': 'Van McCoy The Soul City Symphony', 'pos': 6}

{'year': '1975', 'week': '1975-06-25', 'source_of_genres': '', 'track': 'Listen To What The Man Said', 'genres': [], 'artist': 'Wings', 'spotID': '2Ri2AwoGX3yllyllN5gy9g', 'pos': 7}

{'year': '1975', 'week': '1975-06-25', 'source_of_genres': '', 'track': 'Magic', 'genres': [], 'artist': 'Pilot', 's

In [7]:
hot100s

[{'artist': 'Captain & Tennille',
  'genres': [],
  'pos': 1,
  'source_of_genres': '',
  'spotID': '56ZPl9gFBZxPfRBOBqHfer',
  'track': 'Love Will Keep Us Together',
  'week': '1975-06-25',
  'year': '1975'},
 {'artist': 'Linda Ronstadt',
  'genres': ['country rock'],
  'pos': 2,
  'source_of_genres': 'spotify',
  'spotID': '2MCiAzjR2Uo9cZUnWhGIRI',
  'track': 'When Will I Be Loved',
  'week': '1975-06-25',
  'year': '1975'},
 {'artist': 'Michael Murphey',
  'genres': [],
  'pos': 3,
  'source_of_genres': '',
  'spotID': '4rHOEGE0pH206Q987k7MqF',
  'track': 'Wildfire',
  'week': '1975-06-25',
  'year': '1975'},
 {'artist': 'Jessi Colter',
  'genres': [],
  'pos': 4,
  'source_of_genres': '',
  'spotID': '2G9HPwBaHica41sMtQ7QvX',
  'track': "I'm Not Lisa",
  'week': '1975-06-25',
  'year': '1975'},
 {'artist': 'Major Harris',
  'genres': ['r-b-hip-hop-songs'],
  'pos': 5,
  'source_of_genres': 'billboard',
  'spotID': '5hWHUu9qZXrixYIxoQt7Xm',
  'track': "Love Won't Let Me Wait",
  'we

In [446]:
## There was a problem with wednesdays[62]
# Out[452]:
# '1960-12-28'

[{'artist': [], 'genre': 'pop-songs'},
 {'artist': [], 'genre': 'country-songs'},
 {'artist': [], 'genre': 'rock-songs'},
 {'artist': [], 'genre': 'alternative-songs'},
 {'artist': [], 'genre': 'r-b-hip-hop-songs'},
 {'artist': [], 'genre': 'rap-song'},
 {'artist': [], 'genre': 'dance-electronic-songs'},
 {'artist': [], 'genre': 'latin-songs'},
 {'artist': [], 'genre': 'christian-songs'},
 {'artist': [], 'genre': 'gospel-songs'},
 {'artist': [], 'genre': 'hot-holiday-songs'}]

In [444]:
hotgenres100s

In [2]:
import calendar
import datetime
cal = calendar.Calendar(2)
wednesdays = []
for year in range(1960, 2016):
    for month in range(1, 13):
        for item in cal.monthdatescalendar(year, month):
            wednesdays.append(datetime.datetime.strftime(item[0], '%Y-%m-%d'))
list(set(wednesdays)).sort()

In [16]:
wednesdays[966]

'1975-06-25'

In [449]:
len(wednesdays)

3498